In [15]:
import pandas as pd

In [25]:
import logging

if __name__ == "__main__":
    logging.basicConfig(
        level=logging.INFO,
        force=True,
    )

log = logging.getLogger(__name__)


In [27]:
def convert_to_parquet(stem: str):
    df = pd.read_csv(
        f'data/{stem}.csv',
        engine='pyarrow',
    )

    log.info('read %s.csv: %i rows, %i columns', stem, len(df), len(df.columns))

    df.to_parquet(
        path=f"data/{stem}.parquet.snappy",
        engine="pyarrow",
        compression="snappy",
    )

In [ ]:
def load_dataset(stem: str) -> pd.DataFrame:
    path = f"data/{stem}.parquet.snappy"
    return pd.read_parquet(path)

In [ ]:
if __name__ == "__main__":
    convert_to_parquet('articles')
    convert_to_parquet('customers')
    convert_to_parquet('transactions_train')